## DFS
深度搜索的程序模版：

```python
def func():
    res=list()
    
    def dfs(idx, path):
        '''
        dfs函数，因为是闭包，可以直接访问外部res
        :param idx: 对于全局来说，dfs所在的位置
        :param path: 目前为止dfs已探索的路径
        :return:
        '''
        pass
        dfs(idx + 1, path + something)    # 若path是地址引用，需要改成path[:]
        pass

    init_path = None  # 初始探索路径，一般为0，空字串或空列表
    dfs(0, init_path)
    return res
```

个人习惯把```dfs(idx, path)```写在外部函数中构成闭包，这样做的好处是可以减少```dfs```函数的参数量。需要注意的有两点：
- 如果要在```dfs```函数中修改```res```的话，```res```必须是一个地址引用，如```list```；
- 若```path```是一个地址引用，在递归调用```dfs```时需要传入一个副本```path[:]```。

[Partition to K Equal Sum Subsets](https://leetcode.com/problems/partition-to-k-equal-sum-subsets/)。给一数组与一个正整数$k$，判断该数组能否被划分成和相等的$k$组数。[Matchsticks to Square](https://leetcode.com/problems/matchsticks-to-square/)。给若干火柴棍，以数组表示，每个元素表示火柴棍的长度，判断这些火柴棍能否拼成一个正方形。

思路：虽然该题并不需要求出一个配置，但是还是通过DFS来做。首先易得每一组的和为$sum/k$，若数组中存在比该值大或者除不尽的情况则直接返回False。将数组排序，并初始化$k$个空数组，由大往小开始填充。第二题是第一题$k=4$时的特殊情况。

In [8]:
def canPartitionKSubsets(nums, k: int) -> bool:
    target, rem = divmod(sum(nums), k)
    if rem:
        return False

    nums.sort()
    if nums[-1] > target:
        return False
    while nums and nums[-1] == target:
        nums.pop()
        k -= 1

    init_group = [0]*k

    def dfs(group):
        if not nums:
            return True

        cur_num = nums.pop()
        for i in range(k):
            if group[i]+cur_num <= target:
                group[i] += cur_num
                if dfs(group):
                    return True
                group[i] -= cur_num
        nums.append(cur_num)
        return False

    return dfs(init_group)

[Unique Paths III](https://leetcode.com/problems/unique-paths-iii/)。给以迷宫，其中$1$表示起点，$2$表示终点，$0$表示可通行，$-1$表示障碍物。求从起点到终点并访问一次所有$0$的位置，有多少种走法。

思路：常规的DFS题。首先统计需要访问的点(0)的数目，然后找到起点开始dfs。

In [1]:
def uniquePathsIII(grid):
    rows, cols = len(grid), len(grid[0])
    cnt = 0
    for row in range(rows):
        for col in range(cols):
            if grid[row][col] == 0:
                cnt += 1

    res = 0
    visited = [[False]*cols for _ in range(rows)]

    def dfs(row, col, path):
        nonlocal res

        if row < 0 or row >= rows or col < 0 or col >= cols \
                or grid[row][col] == -1 or visited[row][col]:
            return
        if grid[row][col] == 2:
            if path == cnt+1:
                res += 1
            return

        directions = [(0, 1), (0, -1), (-1, 0), (1, 0)]
        visited[row][col] = True
        for x, y in directions:
            dfs(row+x, col+y, path+1)
        visited[row][col] = False

    for row in range(rows):
        for col in range(cols):
            if grid[row][col] == 1:
                dfs(row, col, 0)
                return res

[Palindrome Partitioning](https://leetcode.com/problems/palindrome-partitioning/)。给一字串，对其进行分割，使得每一段都是一个回文串。求所有可能的分割。

思路：首先，字串所有的分割可以使用DFS。但是要保证每段分割都是回文，需要在每次分割时做一次判断。令DFS中的```idx```为当前分割的起点，遍历所有可能的切分点```i```，当```s[idx:i+1]```是回文时才做进一步的DFS。

In [5]:
def partition(s: str):
    res = list()
    n = len(s)

    def isPalindrome(s):
        if not s or len(s) == 1:
            return True
        else:
            return s[0] == s[-1] and isPalindrome(s[1:-1])

    def dfs(idx, path):
        if idx >= n:
            res.append(path)
            return

        for i in range(idx, n):
            if isPalindrome(s[idx:i+1]):
                dfs(i+1, path+[s[idx:i+1]])

    dfs(0, list())
    return res

[['a', 'a', 'b'], ['aa', 'b']]

[Beautiful Arrangement](https://leetcode.com/problems/beautiful-arrangement/)。给$N$个自然数，对其进行排列，要求对任意位置都至少满足以下一条性质，其中索引从$1$开始。
- ```arr[i]```可以被```i```整除；
- 或者```i```可以被```arr[i]```整除。

思路：DFS，每次选取一个未使用过且符合条件的数字加到```path```中，需要维护一个访问数组。

In [9]:
def countArrangement(N: int) -> int:
    visited = [False]*(N+1)
    res = list()

    def dfs(idx, path):
        if idx == (N+1):
            res.append(path)
            return

        for num in range(1, N+1):
            if not visited[num] and (num % idx == 0 or idx % num == 0):
                visited[num] = True
                dfs(idx+1, path+[num])
                visited[num] = False

    dfs(1, list())
    return len(res)

8

[Letter Case Permutation](https://leetcode.com/problems/letter-case-permutation/)。给一包含数字和字母的字串，其中字串中的字母可以随机更换大小写，求出所有可能的变换。

思路：DFS，```idx```表示工作指针，首先判断```s[idx]```是否是字母，若是字母，则分别对大小写进行递归。

In [18]:
def letterCasePermutation(S: str):
    n = len(S)
    res = list()

    def dfs(idx, path):
        if idx == n:
            res.append(''.join(path))
            return

        if S[idx].isalpha():
            dfs(idx+1, path+[S[idx].upper()])
            dfs(idx+1, path+[S[idx].lower()])
        else:
            dfs(idx+1, path+[S[idx]])

    dfs(0, list())
    return res

['A1B2', 'A1b2', 'a1B2', 'a1b2']

[Most Stones Removed with Same Row or Column](https://leetcode.com/problems/most-stones-removed-with-same-row-or-column/)。恕我直言，原题有歧义，改述如下。一二维平面上放置若干个石头，选一个起点出发，每次移动只能沿轴向移动到下一个石头上，不能往复移动。问最多能走多少步。

思路：相比于直接求能走多少步，另外一个思路比较简单。矩阵中的石头能通过轴向线段连接起来形成联通分量，最多能走的步数=石头数量-联通分量数。在轴向上做DFS可以遍历到一个联通分量的所有点。

In [1]:
def removeStones(stones) -> int:
    n = len(stones)
    visited = set()

    def dfs(i, j):
        if (i, j) in visited:
            return 0

        visited.add((i, j))
        for stone in stones:
            if stone[0] == i or stone[1] == j:
                dfs(stone[0], stone[1])
        return 1

    return n-sum([dfs(i, j) for i, j in stones])

5

[机器人的运动范围](https://www.acwing.com/problem/content/22/)。给一指定行列数的方格矩形，从原点$(0,0)$出发。再给定一阈值，若某坐标的行列数字之和超过该阈值，则无法进入该坐标。如$threshold=18$，则不能进入$(35,38)$。

思路：DFS。维护一个访问矩阵，可以进行DFS有两个条件，未被访问并且坐标数字和小于阈值。

In [6]:
def movingCount(threshold, rows, cols):
    res = 0
    visited = [[False]*cols for _ in range(rows)]

    def isValid(i, j):
        return True if sum(map(int, str(i)+str(j))) <= threshold else False

    def dfs(i, j):
        nonlocal res
        if 0 <= i < rows and 0 <= j < cols and not visited[i][j] and isValid(i, j):
            visited[i][j] = True
            res += 1
            dfs(i+1, j)
            dfs(i-1, j)
            dfs(i, j+1)
            dfs(i, j-1)

    dfs(0, 0)
    return res

0

[N-Queens](https://leetcode.com/problems/n-queens/)。输出N皇后的所有解。

思路：经典八皇后问题。暴力解，每次在一行放置一个皇后，在放置下一行的皇后时，需要判断不同列与不同的对角线，即左上对角线与右上对角线。

In [ ]:
def solveNQueens(n: int):
    path = [['.'] * n for _ in range(n)]
    used = [False] * n    # 列的占用标记

    def isValid(row, col):
        left = right = col

        for i in range(row - 1, -1, -1):
            if left - 1 >= 0:  # 左上斜线
                left -= 1
                if path[i][left] == 'Q':
                    return False

            if right + 1 < n:  # 右上斜线
                right += 1
                if path[i][right] == 'Q':
                    return False

        return True

    res = list()
    import copy

    def dfs(idx):
        '''
        在行(row)上进行dfs，idx表示行号
        '''
        if idx == n:
            res.append(copy.deepcopy(path))
            return

        for col in range(n):
            if not used[col] and isValid(idx, col):
                used[col] = True
                path[idx][col] = 'Q'
                dfs(idx + 1)
                used[col] = False
                path[idx][col] = '.'

    dfs(0)
    ret = [[''.join(row) for row in mat] for mat in res]
    return ret

[Surrounded Regions](https://leetcode.com/problems/surrounded-regions/)。给一个由'X'和'O'构成的矩阵，将所有被'X'包围的'O'转换成'X'。

思路：将所有'X'包围的'O'转换成'X'，那么不需要转换的肯定是与边界相连的'O'。使用DFS将所有与边界相连的'O'置成'#'保护起来，然后将所有的'O'转成'X'，再将'#'还原成'O'。

In [ ]:
def solve(board) -> None:
    if not board:
        return None
    rows, cols = len(board), len(board[0])

    def dfs(row, col):
        board[row][col] = '#'
        if row > 0 and board[row-1][col] == 'O':
            dfs(row-1, col)
        if row < rows-1 and board[row+1][col] == 'O':
            dfs(row+1, col)
        if col > 0 and board[row][col-1] == 'O':
            dfs(row, col-1)
        if col < cols-1 and board[row][col+1] == 'O':
            dfs(row, col+1)

    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 'O' and (row == 0 or row == rows-1 or col == 0 or col == cols-1):
                dfs(row, col)

    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 'O':
                board[row][col] = 'X'

    for row in range(rows):
        for col in range(cols):
            if board[row][col] == '#':
                board[row][col] = 'O'

[Restore IP Addresses](https://leetcode.com/problems/restore-ip-addresses/)。给定一只含数字的字符串，求其能生成的所有合法IP地址。

思路：逻辑实现。

In [7]:
def restoreIpAddresses(s: str):
    n=len(s)
    res=list()
    
    def dfs(idx,path):
        thresh=(4-len(path))*3
        if n-idx>thresh:
            return
        
        if idx==n and len(path)==4:
            res.append('.'.join(path))
            return
        
        for i in range(idx+1,idx+4):
            if i<=n:
                cur_num=int(s[idx:i])
                if str(cur_num)==s[idx:i] and cur_num<=255:
                    dfs(i,path+[s[idx:i]])
                    
    dfs(0,list())
    return res

['255.255.11.135', '255.255.111.35']

[黑白棋](https://www.nowcoder.com/questionTerminal/a0feb0696e2043a5b3b0779fa861b64a?f=discussion&toCommentId=6271446)。给一$8\times8$的棋盘，$0$表示空白，$1$表示黑色，$2$表示白色。当两个同色棋子之间(八个方向均可)夹有对方棋子时，可以将对方棋子翻转成己方颜色。白色先手，求在白色方下完$N$手后棋盘内可能有的最多的白棋数。

思路：DFS。注意白方先手，黑方后手，在每次dfs时需要转换变量。该题有个坑，用例给定的初始棋盘是可以翻转的，理论上来说需要先对初始棋盘进行翻转再去落子，因此最后一个用例(满棋盘)需要特殊处理。

In [ ]:
import sys

hands = int(sys.stdin.readline().strip())

board = [list(map(int, sys.stdin.readline().strip().split()))
         for _ in range(8)]

res = 0
for row in range(8):
    for col in range(8):
        if board[row][col] == 2:
            res += 1

directions = [[-1, 1], [0, 1], [1, 1], [1, 0],
              [1, -1], [0, -1], [-1, -1], [-1, 0]]  # 八个方向基向量


def flip(board, coords):
    '''
    翻转棋盘
    :param board:
    :param coords: 需要翻转的棋子坐标
    :return: 翻转的棋子数量
    '''
    for row, col in coords:
        board[row][col] = board[row][col] % 2 + 1
    return len(coords)


def valid(board, row, col, color):
    '''
    检查当前坐标落子的有效性
    :param board: 棋盘
    :param row,col: 落子坐标
    :param color: 落子颜色
    :return: 可以被翻转棋子的坐标列表
    '''
    res = list()

    if board[row][col] != 0:  # 已存在棋子
        return res

    for dx, dy in directions:
        single_line_res = list()  # 单方向扫描到的异色棋子
        for t in range(1, 9):  # 基向量的倍率
            s_row, s_col = row + t * dx, col + t * dy

            # 该方向走到边界或遇到空
            if s_row < 0 or s_row > 7 \
                    or s_col < 0 or s_col > 7 \
                    or board[s_row][s_col] == 0:
                break
            # 遇到同色棋子
            elif board[s_row][s_col] == color:
                res.extend(single_line_res)
                single_line_res = list()  # 清空
                break
            # 异色棋子
            else:
                single_line_res.append([s_row, s_col])

    return res


def dfs(step, cur_res, color, board):
    global res, hands
    res = max(cur_res, res)

    if step == hands and color == 1:
        return

    for row in range(8):
        for col in range(8):
            coords = valid(board, row, col, color)
            if coords:
                board[row][col] = color  # 落子
                cur_res_bak = cur_res  # 备份用于状态还原

                if color == 2:
                    cur_res += 1
                    cur_res += flip(board, coords)
                else:
                    cur_res -= flip(board, coords)

                next_step = step + color % 2
                next_color = color % 2 + 1
                dfs(next_step, cur_res, next_color, board)

                # 状态还原
                board[row][col] = 0
                flip(board, coords)
                cur_res = cur_res_bak

    return


dfs(1, res, 2, board)
print(res)

## BFS
BFS与DFS都属于暴力解法，但是BFS可用于寻找最短路径。

[Number of Islands](https://leetcode.com/problems/number-of-islands/)。求数字矩阵中数字'1'构成的孤岛数量。

思路：设置访问矩阵，线性扫描原矩阵。当找到一个$1$时，就作为起点开始做BFS。

In [1]:
def numIslands(grid) -> int:
    if not grid:
        return 0

    rows, cols = len(grid), len(grid[0])
    visited = [[False]*cols for _ in range(rows)]

    def bfs(row, col):
        if row < 0 or row >= rows or col < 0 or col >= cols or grid[row][col] == '0' or visited[row][col]:
            return

        visited[row][col] = True

        bfs(row-1, col)
        bfs(row+1, col)
        bfs(row, col-1)
        bfs(row, col+1)

    res = 0
    for row in range(rows):
        for col in range(cols):
            if grid[row][col] == '0' or visited[row][col]:
                continue

            bfs(row, col)
            res += 1

    return res

[Max Area of Island](https://leetcode.com/problems/max-area-of-island/)。给一$01$矩阵，$0$代表海洋而$1$代表陆地，上下左右相连的陆地构成岛屿。求最大岛屿的最大面积。

思路：设置访问矩阵，线性扫描原矩阵，当遇到$1$时，就作为起点做BFS。

In [4]:
def maxAreaOfIsland(grid) -> int:
    rows, cols = len(grid), len(grid[0])
    visited = [[False]*cols for _ in range(rows)]
    res = 0
    offset = [(-1, 0), (1, 0), (0, -1), (0, 1)]    # 上下左右的偏移量

    for row in range(rows):
        for col in range(cols):
            if grid[row][col] == 0 or visited[row][col]:
                continue

            q = [(row, col)]
            cur_res = 0
            while q:
                vis_row, vis_col = q.pop(0)
                if grid[vis_row][vis_col] == 0 or visited[vis_row][vis_col]:
                    continue

                visited[vis_row][vis_col] = True
                cur_res += 1
                for delta_x, delta_y in offset:
                    nei_row, nei_col = vis_row+delta_x, vis_col+delta_y
                    if 0 <= nei_row < rows and 0 <= nei_col < cols:
                        q.append((nei_row, nei_col))

            res = max(res, cur_res)

    return res

6

[Shortest Bridge](https://leetcode.com/problems/shortest-bridge/)。给一$01$矩阵，$0$代表海洋而$1$代表陆地，矩阵中存在两个岛屿，求两个岛屿的最短距离。

思路：DFS+BFS。首先通过遍历找到一个岛屿，将该岛屿的所有单元格全存入队列等待BFS，在BFS过程中维护一个距离变量即可。如果遇到未访问的$1$，则说明到达了新岛屿。

In [27]:
def shortestBridge(A) -> int:
    rows, cols = len(A), len(A[0])
    visited = [[False]*cols for _ in range(rows)]
    q = list()

    def dfs(i, j):
        if 0 <= i < rows and 0 <= j < cols and not visited[i][j] and A[i][j] == 1:
            visited[i][j] = True
            q.append((i, j, 0))
            for x, y in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
                dfs(x, y)

    def get_a_island():
        for row in range(rows):
            for col in range(cols):
                if A[row][col] == 1:
                    dfs(row, col)
                    return

    get_a_island()

    while q:
        i, j, step = q.pop(0)
        for x, y in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
            if 0 <= x < rows and 0 <= y < cols and not visited[x][y]:
                if A[x][y] == 1:
                    return step
                else:
                    visited[x][y] = True
                    q.append((x, y, step+1))

0

[Letter Combinations of a Phone Number](https://leetcode.com/problems/letter-combinations-of-a-phone-number/)。九宫格键盘能实现的字符串，给定一组数字，求出九宫格键盘按该序列能拼出的字符串。

思路：初始答案肯定是空字串，每按一个键，就会在空字串上增加若干单字串答案。在已有答案上的修改是解题的关键。

In [1]:
def letterCombinations(digits: str):
    if not digits:
        return list()
    
    lookup = ["", "", "abc", "def", "ghi", "jkl", "mno", "pqrs", "tuv", "wxyz"]
    res = ['']

    for ch in digits:
        bak_chs = list(lookup[int(ch)])
        cur_res = list()
        for s in res:
            for c in bak_chs:
                cur_res.append(s+c)

        res = cur_res

    return res

['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']

[Word Ladder](https://leetcode.com/problems/word-ladder/)。给定一个源单词与目标单词还有一个词典，每次允许变换一个字母，求从源单词变为目标单词需要的步数。

思路：BFS，每次改变源单词的一个字母，找到对应的字典中的词，并维护一个访问状态。

In [ ]:
def ladderLength(beginWord: str, endWord: str, wordList) -> int:
    wordList = set(wordList)
    n = len(wordList)
    q = [(beginWord, 1)]
    visited = set()

    while q:
        word, step = q.pop(0)
        if word == endWord:
            return step
        if word in visited:
            continue

        visited.add(word)
        for idx in range(len(word)):
            for ch_idx in range(26):
                next_word = word[:idx]+chr(97+ch_idx)+word[idx+1:]
                if next_word in wordList:
                    q.append((next_word, step+1))

    return 0

[Open the Lock](https://leetcode.com/problems/open-the-lock/)。四位旋转密码锁，其中有几个位置是死区(deadend)，一旦转到该位置锁会锁死。给定一系列死区，初始状态为```0000```，问开锁需要转几次旋钮。

思路：求最短路径使用BFS。因为密码锁有$4$位，对于每一个状态，可以有$8$种邻接状态。

In [16]:
def openLock(deadends, target: str) -> int:
    visited = set()
    deadends = set(deadends)
    q = [('0000', 0)]

    while q:
        vis_stat, cnt = q.pop(0)
        if vis_stat == target:
            return cnt

        if vis_stat in visited or vis_stat in deadends:
            continue

        visited.add(vis_stat)
        for idx in range(4):
            pre = vis_stat[:idx] + \
                str((int(vis_stat[idx])-1) % 10)+vis_stat[idx+1:]
            post = vis_stat[:idx] + \
                str((int(vis_stat[idx])+1) % 10)+vis_stat[idx+1:]
            if pre not in visited and pre not in deadends:
                q.append((pre, cnt+1))
            if post not in visited and post not in deadends:
                q.append((post, cnt+1))

    return -1

6

[Keys and Rooms](https://leetcode.com/problems/keys-and-rooms/)。给一个列表，每个元素表示一个房间，每个房间里有若干把钥匙。第一个房间未锁，判断从第一个房间开始，能否成功打开所有房间。

思路：DFS或BFS，BFS可用队列实现优先考虑BFS。维护访问状态，从第一个房间进入拿钥匙加入队列，并标记访问；依次取出每个房间的钥匙即可。最后判断所有房间是否都已访问过。

In [2]:
def canVisitAllRooms(rooms) -> bool:
    visited = [False]*len(rooms)
    q = rooms[0]
    visited[0] = True

    while q:
        vis_room = q.pop(0)
        visited[vis_room] = True
        for key in rooms[vis_room]:
            if not visited[key]:
                q.append(key)

    return all(visited)

True

[Minesweeper](https://leetcode.com/problems/minesweeper/)。扫雷模拟，'M'表示地雷，'E'表示未点击方块，'B'表示空白块，$[1-8]$的数字表示周围的地雷数，'X'表示踩雷。给一个游戏盘与一个点击位置，返回点击后的状态。

思路：逻辑实现，递归。如果点击到'B'与数字，无改变；踩到雷，改成'X'；点击'E'，若周围没有雷，则递归点击周围所有格子，否则改成数字。难点在于周围八个点的存在性判断，最蠢的方法就是写多个判断条件。

In [6]:
def updateBoard(board, click):
    rows, cols = len(board), len(board[0])

    def bfs(row, col):
        if board[row][col] == 'M':
            board[row][col] = 'X'
            return
        elif board[row][col] == 'B':
            return
        else:
            n_mine = 0
            for d_x in (-1, 0, 1):
                for d_y in (-1, 0, 1):
                    new_row, new_col = row+d_x, col+d_y
                    if new_row in range(rows) and new_col in range(cols) and board[new_row][new_col] == 'M':
                        n_mine += 1
            if n_mine > 0:
                board[row][col] = str(n_mine)
            else:
                board[row][col] = 'B'
                for d_x in (-1, 0, 1):
                    for d_y in (-1, 0, 1):
                        new_row, new_col = row+d_x, col+d_y
                        if new_row in range(rows) and new_col in range(cols):
                            bfs(new_row, new_col)

    bfs(click[0], click[1])
    return board

[['B', '1', 'E', '1', 'B'],
 ['B', '1', 'M', '1', 'B'],
 ['B', '1', '1', '1', 'B'],
 ['B', 'B', 'B', 'B', 'B']]

[01 Matrix](https://leetcode.com/problems/01-matrix/)。给一个只含$0$和$1$的矩阵，返回一个矩阵，该矩阵记录的是相应位置元素离最近$0$元素的距离，距离只算上下左右，不能斜着走。

思路：BFS。以每一个非零位置出发做BFS，上下左右存在$0$即刻返回。

In [14]:
def updateMatrix(matrix):
    rows, cols = len(matrix), len(matrix[0])

    def bfs(row, col):
        q = [(row, col)]
        res = 0
        while q:
            level_size = len(q)
            res += 1
            for _ in range(level_size):
                vis_row, vis_col = q.pop(0)
                for d_x, d_y in zip([0, 0, -1, 1], [1, -1, 0, 0]):
                    new_row, new_col = vis_row+d_x, vis_col+d_y
                    if new_row in range(rows) and new_col in range(cols):
                        if matrix[new_row][new_col] == 0:
                            return res
                        else:
                            q.append((new_row, new_col))

    for row in range(rows):
        for col in range(cols):
            if matrix[row][col] != 0:
                matrix[row][col] = bfs(row, col)

    return matrix

[[0, 0, 0], [0, 1, 0], [1, 2, 1]]